In [11]:
from core.DataLoader import *
from core.models.MLP import * 
from core.models.GAM import *
from core.models.SVM import *
from core.models.EBM import *
from core.models.RandomForest import *
from core.Visualizer import *
from get_variable_name import *
import itertools
from multiprocessing import Pool
import json

def read_data(engine='postgres'):
    dl = DataLoader(engine)
    one_file_dss = dl.get_one_file_ds()
    clustered_file_ds = dl.get_clustered_files_ds()
    all_file_ds = dl.get_all_files_ds()
    return one_file_dss, clustered_file_ds, all_file_ds

class classification_model_runner():
    def __init__(self, original_df):
        self.df = original_df
        self.models = {}
        self.model_acc = {}

    def set_dataframe(self, df):
        self.df = df

    def prepare_features(self, train_features, target_feature='optimal_decision', cost_features=DataLoader().regression_targets):
        self.train_features = train_features
        X = self.df[train_features]
        y = self.df[target_feature]
        X_costs = self.df[cost_features]

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
        X_train_costs, _, _, _, = train_test_split(X_costs, y, train_size=0.8, random_state=1)
        X_train, X_test, y_train, y_test = \
            X_train.to_numpy(),X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

        X_train_weights = calculate_importance_from_costs(X_train_costs.to_numpy())

        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)

        # scaler = preprocessing.StandardScaler().fit(X_test)
        X_test = scaler.transform(X_test)

        X_train_weights = preprocessing.MinMaxScaler().fit_transform(X_train_weights.reshape(-1,1))

        self.X_train = X_train
        self.X_test = X_test
        self.X_test = X_test
        self.X_train_weights = X_train_weights
        self.X_scaler = scaler
        
        self.y_train = y_train
        self.y_test = y_test
        

    def set_model_list(self, model_name_list):
        self.model_name_list = model_name_list

    def train_models(self):
        for model_name in self.model_name_list:
            if model_name == MLPClassifier:
                clf = model_name(n_features=len(self.train_features))
            elif model_name == sk_nn.MLPClassifier:
                clf = model_name(hidden_layer_sizes=(100,100,100,100))
            else:
                clf = model_name()
            clf = clf.fit(self.X_train, self.y_train)
            acc = clf.score(self.X_test, self.y_test)
#             print(f"Accuray of {model_name}: {acc}")
            # if len(features) == 2:
            #     plot_2d_decision_boundaries(clf, X_train, X_train_costs, y_train, title=f'{model} on {i}-th one table pair')
            self.models[model_name] = clf
            self.model_acc[model_name] = acc    

In [12]:
one_file_dss, clustered_file_ds, all_file_ds = read_data()
# all_dfs = [ds for ds in one_file_dss] + [clustered_file_ds, all_file_ds]
all_dfs = [all_file_ds]

In [13]:
all_features = DataLoader().all_features
model_name_list=[sk_nn.MLPClassifier, m_RandomForestClassifier,
                          SVMClassifier, LinearGAMClassifier, EBMClassifier]

# Features combo

In [14]:
from multiprocessing import Pool
global performances


def eval_features(train_features):
    global performaces
    accs = []
    for ds in all_dfs:
        runner = classification_model_runner(ds)
        runner.prepare_features(list(train_features))
        runner.set_model_list(model_name_list=model_name_list)
        runner.train_models()
        accs.append(np.average(list(runner.model_acc.values())))
    performances[tuple(train_features)] = accs

for n in [2]:
    performances = {}
    args = []

    for train_features in tqdm(list(itertools.combinations(all_features, n))):
        args.append(list(train_features))
        
    with Pool(50) as p:
        p.map(eval_features, args)
    with open(f'all_perfomances_{n}.json', 'w') as fp:
        json.dump(performances, fp)

100%|██████████| 455/455 [00:00<00:00, 225287.25it/s]


<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
<class 'core.models.RandomForest.m_RandomForestClassifier'>
<class 'core.models.SVM.SVMClassifier'>
<class 'core.models.GAM.LinearGAMClassifier'>
<class 'core.models.EBM.EBMClassifier'>


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:933: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  "argument is ignored.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/interpret/glassbox/ebm/ebm.py:801: UserWarning: Multiclass is still experimental. Subject to change per release.
  warn("Multiclass is still experimental. Subject to change per release.")


{('left_cardinality_ratio',
  'left_cardinality',
  'base_cardinality'): [0.7540833333333333]}

# Read and eval

In [ ]:
# with open('all_perfomances.json') as json_file:
#     performances = json.load(json_file)